In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


print ('Library loaded')

Library loaded


In [3]:
base = r"E:\Skripsi Dion\Dataset Skin Cancer (Harvard Database)"

# folders_to_create = ['HAM10000_all-images', 'HAM10000']
folders_to_create = ["HAM10000_224new6"]


def create_folders(base_path, folders):
    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder '{folder}' sudah dibuat.")
        else:
            print(f"Folder '{folder}' sudah ada.")


create_folders(base, folders_to_create)

Folder 'HAM10000_224new6' sudah dibuat.


In [4]:
base = r'E:\Skripsi Dion\Dataset Skin Cancer (Harvard Database)'
data_pd = pd.read_csv('E:\Skripsi Dion\Dataset Skin Cancer (Harvard Database)\HAM10000_metadata.csv')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [5]:
train_dir = os.path.join(base, "HAM10000_224new6", "train_dir")
test_dir = os.path.join(base, "HAM10000_224new6", "test_dir")

In [6]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization
lesion_id,,,,,,
HAM_0000000,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1


In [7]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [8]:
#Memeriksa apakah terdapat gambar yang memiliki duplikat
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no'
    else:
        return 'duplicates'

In [9]:
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates


In [10]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

In [11]:
#Menghitung banyaknya gambar perkelas sebelum di augmentasi
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

data_pd.dx=data_pd.dx.astype('category',copy=True)
data_pd['label']=data_pd.dx.cat.codes # Convert the labels to numbers
data_pd['lesion_type']= data_pd.dx.map(lesion_type_dict)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate,label,lesion_type
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates,2,Benign keratosis


In [12]:
class_counts = data_pd['lesion_type'].value_counts()
print(class_counts)

lesion_type
Melanocytic nevi        6705
Melanoma                1113
Benign keratosis        1099
Basal cell carcinoma     514
Actinic keratoses        327
Vascular lesions         142
Dermatofibroma           115
Name: count, dtype: int64


In [13]:
#Split test data 15%
train, test_df = train_test_split(df_count, test_size=0.20, stratify=df_count['dx'])

In [14]:
#Memeriksa tiap gambar masuk kedalam jenis train atau test
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate,label,lesion_type,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates,2,Benign keratosis,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates,2,Benign keratosis,train


In [15]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
5693,HAM_0001293,ISIC_0030062,nv,follow_up,45.0,female,abdomen,no
5764,HAM_0005394,ISIC_0026903,nv,follow_up,60.0,female,trunk,no
6042,HAM_0001521,ISIC_0031860,nv,follow_up,40.0,male,foot,no
4755,HAM_0004241,ISIC_0024616,nv,follow_up,35.0,male,lower extremity,no
6329,HAM_0004650,ISIC_0028149,nv,follow_up,35.0,female,lower extremity,no


In [16]:
# Image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [17]:
#Total data untuk train dan test
print(len(train_list))
print(len(test_list))


8912
1103


In [18]:
data_pd.set_index('image_id', inplace=True)

In [19]:
os.mkdir(train_dir)
os.mkdir(test_dir)

In [20]:
classname = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [21]:
for i in classname:
  directory1=train_dir+'/'+i
  directory2=test_dir+'/'+i
  os.mkdir(directory1)
  os.mkdir(directory2)

In [22]:
#Memasukkan gambar dari HAM10000_all-images kedalam train file
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image
    source = os.path.join(base, 'HAM10000_all-images', file_name)

    # copying the image from the source to target file
    target = os.path.join(train_dir, label, file_name)

    shutil.copyfile(source, target)

In [23]:
#Memasukkan gambar dari HAM10000_all-images kedalam test file
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image
    source = os.path.join(base, 'HAM10000_all-images', file_name)

    # copying the image from the source to target file
    target = os.path.join(test_dir, label, file_name)

    shutil.copyfile(source, target)

In [24]:
# Count the number of images for each class in the train file
train_class_counts = train_df['lesion_type'].value_counts()
print("Train File Class Counts:")
print(train_class_counts)


Train File Class Counts:
lesion_type
Melanocytic nevi        5822
Melanoma                1067
Benign keratosis        1011
Basal cell carcinoma     479
Actinic keratoses        297
Vascular lesions         129
Dermatofibroma           107
Name: count, dtype: int64


In [25]:
base = r'E:\Skripsi Dion\Dataset Skin Cancer (Harvard Database)'

In [26]:
classname = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
for img_class in classname:
    aug_dir = os.path.join(base, "HAM10000_224new6", "aug_dir")
    os.mkdir(aug_dir)
    img_dir = os.path.join(aug_dir, "img_dir")
    os.mkdir(img_dir)

    img_list = os.listdir(
        os.path.join(base, "HAM10000_224new6", "train_dir", img_class)
    )
    for file_name in img_list:
        source = os.path.join(
            base, "HAM10000_224new6", "train_dir", img_class, file_name
        )
        target = os.path.join(img_dir, file_name)
        shutil.copyfile(source, target)

    source_path = aug_dir
    save_path = os.path.join(base, "HAM10000_224new6", "train_dir", img_class)
    batch_size = 50

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.05,

    )

    aug_datagen = datagen.flow_from_directory(
        source_path,
        save_to_dir=save_path,
        save_format="jpg",
        target_size=(224, 224),
        batch_size=batch_size,
    )
    ###jelaskan mengapa bisa sampai 6000gambar per kelas
    aug_images = 6000

    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # hapus temporary directory
    shutil.rmtree(aug_dir)

Found 297 images belonging to 1 classes.
Found 479 images belonging to 1 classes.
Found 1011 images belonging to 1 classes.
Found 107 images belonging to 1 classes.
Found 1067 images belonging to 1 classes.
Found 5822 images belonging to 1 classes.
Found 129 images belonging to 1 classes.


In [27]:
# import os
# import numpy as np


# def count_files_and_batches(img_dir, aug_images, batch_size):
#     num_files = len(os.listdir(img_dir))
#     num_batches = int(np.ceil((aug_images - num_files) / batch_size))

#     return num_files, num_batches


# # Contoh penggunaan fungsi untuk menghitung num_files dan num_batches
# img_dir_example = os.path.join(base, "HAM10000_224new6", "train_dir", "bkl")
# aug_images_example = 6000
# batch_size_example = 50

# num_files_example, num_batches_example = count_files_and_batches(
#     img_dir_example, aug_images_example, batch_size_example
# )

# print("num_files_example adalah:", num_files_example)
# print("num_batches_example adalah:", num_batches_example)

# # Melanocytic nevi        5822 -> 6022 nv
# # Melanoma                1067 -> 5885 mel 6000-1067=4933, 98 99
# # Benign keratosis        1011 -> 5855 bkl
# # Basal cell carcinoma     479 -> 5798 bcc
# # Actinic keratoses        297 -> 5990 akiec
# # Vascular lesions         129 -> 5210 vasc
# # Dermatofibroma           107 -> 4330 df

In [28]:
# print("Jumlah gambar sebelum augmentasi:", num_files_example)
# print("Jumlah batches:", num_batches_example)
# print("Total gambar teraugmentasi seharusnya:",num_files_example+(batch_size_example * num_batches_example))

In [29]:
# img_dir = os.path.join(base, "HAM10000_224new6", "train_dir", "bkl")

# num_files = len(os.listdir(img_dir))
# aug_images = 6000
# batch_size = 50

# # Calculate the number of batches needed to reach the target
# num_batches = int(np.ceil((aug_images - num_files) / batch_size))

# # Loop through the batches and generate augmented images
# for i in range(0, num_batches):
#     images, labels = next(aug_datagen)

In [30]:
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode="nearest",
# )

# aug_datagen = datagen.flow_from_directory(
#     source_path,
#     save_to_dir=save_path,
#     save_format="jpg",
#     target_size=(224, 224),
#     batch_size=batch_size,
# )
# #jelaskan mengapa bisa sampai 6000gambar per kelas
# aug_images = 6000

# num_files = len(os.listdir(img_dir))
# num_batches = int(np.ceil((aug_images - num_files) / batch_size))

# for i in range(0, num_batches):
#     images, labels = next(aug_datagen)

# # hapus temporary directory
# shutil.rmtree(aug_dir)

In [31]:
def count_images_per_class(directory):
    class_counts = {}

    for img_class in os.listdir(directory):
        class_path = os.path.join(directory, img_class)
        if os.path.isdir(class_path):
            num_images = len(os.listdir(class_path))
            class_counts[img_class] = num_images

    return class_counts


augmented_train_dir = os.path.join(base, "HAM10000_224new6", "train_dir")

augmented_class_counts = count_images_per_class(augmented_train_dir)

print("Banyak gambar per kelas pada train_df setelah augmentasi:")
for img_class, count in augmented_class_counts.items():
    print(f"{img_class}: {count} images")

Banyak gambar per kelas pada train_df setelah augmentasi:
akiec: 5990 images
bcc: 5798 images
bkl: 5855 images
df: 4330 images
mel: 5885 images
nv: 6022 images
vasc: 5210 images


In [32]:
test_class_counts = test_df["dx"].value_counts()
print("Banyak gambar per kelas pada test_df:")
print(test_class_counts)

Banyak gambar per kelas pada test_df:
dx
nv       883
bkl       88
mel       46
bcc       35
akiec     30
vasc      13
df         8
Name: count, dtype: int64
